<a href="https://colab.research.google.com/github/ilitteri/7506-Datos/blob/main/tp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
#from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [3]:
df_business = pd.read_csv('/content/drive/MyDrive/Dataset/business.csv')

In [4]:
df_business.to_parquet('business.parquet')

In [5]:
del(df_business)

In [6]:
df_review = pd.read_csv('/content/drive/MyDrive/Dataset/review.csv')
df_review = df_review.sample(frac=.3)

In [7]:
df_review.to_parquet('review.parquet')

In [8]:
del(df_review)

In [9]:
df_user = pd.read_csv('/content/drive/MyDrive/Dataset/user.csv')
df_user = df_user.sample(frac=.3)

In [10]:
df_user.to_parquet('user.parquet')

In [11]:
del(df_user)

In [12]:
spark = SparkSession.builder.getOrCreate()
spark_context = spark.sparkContext
sql_context = SQLContext(spark_context)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [13]:
df_business = sql_context.read.parquet('/content/business.parquet')
df_review = sql_context.read.parquet('/content/review.parquet')
df_user = sql_context.read.parquet('/content/user.parquet')

In [14]:
rdd_business = df_business.rdd
rdd_review = df_review.rdd
rdd_review = df_review.rdd

# S13

¿Cuál es el restaurante mexicano más cercano a FIUBA? Datos ubicación FIUBA: Latitud:-34.61748 – Longitud:-58.36824.

Notas: para éste ejercicio me topé con la opción de ordenar a con `.takeOrdered()` o `.reduce()` enviándole una función lambda para la comparación. Y ante la duda consulté con mi corrector para ver cuál era mejor. Resolvimos que cualquiera de las dos servía para el caso, y para demostrar ésto corrí `%%timeit` para ambos casos, siendo el resultado el siguiente. Me quedé con `.reduce()` porque vi que Damian en un video dijo que era mejor que usar un `.takeOrdered()`.

<figure>
    <img src="https://i.imgur.com/yn3XSZq.png">
</figure>

In [15]:
import math

from math import radians, cos, sin, asin, sqrt
def distance(lat1, lon1, lat2, lon2):
     
    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = math.radians(lon1)
    lon2 = math.radians(lon2)
    lat1 = math.radians(lat1)
    lat2 = math.radians(lat2)
      
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
 
    c = 2 * asin(sqrt(a))
    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
      
    # calculate the result
    return(c * r)

fiuba = (-34.61748, -58.36824)

locations = rdd_business \
    .filter(lambda row: row.categories and 'Mexican' in row.categories) \
    .map(lambda row: (row.name, distance(fiuba[0], fiuba[1], float(row.latitude), float(row.longitude)))) \
    .reduce(lambda x, y: x if x[1] < y[1] else y)

print("El restaurante mexicano más cercano a FIUBA es:", locations[0])

El restaurante mexicano más cercano a FIUBA es: Waypoint Bar and Grill


# S20

In [16]:
a = rdd_business \
    .filter(lambda row: row.state == 'LA' and row.name == "McDonald's") \
    .distinct() \
    .map(lambda row: ((row.address, row.city), (row.latitude, row.longitude))) \
    .cache()
''' Me di cuenta que se podia hacer cartesian de un RDD consigo mismo
b = rdd \
    .filter(lambda row: row.state == 'LA' and row.name == "McDonald's") \
    .distinct() \
    .map(lambda row: ((row.address, row.city), (row.latitude, row.longitude))) \
    .cache()
'''

cartesian_rdd_between_mcdonalds = a.cartesian(a)

In [17]:
closest_mcdonalds = cartesian_rdd_between_mcdonalds \
    .map(lambda x: ((x[0][0], x[1][0]), distance(x[0][1][0], x[0][1][1], x[1][1][0], x[1][1][1]))) \
    .filter(lambda x: x[0][0] != x[0][1]) \
    .reduce(lambda x, y: x if x[1] < y[1] else y)
closest_mcdonalds

((('1212 Stumpf Blvd', 'Gretna'), ('501 Westbank Expy', 'Gretna')),
 1.1585254095849085)

In [18]:
print(f"Los locales de McDonald's más cercanos en LA tienen dirección {closest_mcdonalds[0][0][0]} y {closest_mcdonalds[0][1][0]} y se encuentran en la ciudad de {closest_mcdonalds[0][0][1]}")

Los locales de McDonald's más cercanos en LA tienen dirección 1212 Stumpf Blvd y 501 Westbank Expy y se encuentran en la ciudad de Gretna


# S17

In [20]:
df_user_mod = df_user.withColumn("yelping_since", df_user.yelping_since.cast("date"))
rdd_user_mod = df_user_mod.rdd

In [21]:
PRESENT_YEAR = 2022
antiguedades = rdd_user_mod \
    .map(lambda row: (row.user_id, (row.name, row.yelping_since))) \
    .cache()

In [22]:
usuarios_cuya_review_mas_reciente_contiene_la_palabra_pizza = rdd_review \
    .filter(lambda row: row.text and row.date and 'pizza' in row.text.lower()) \
    .map(lambda row: (row.user_id, row.date)) \
    .reduceByKey(lambda a, b: a if a > b else b) \
    .cache()

In [23]:
usuario_con_mas_antiguedad_cuya_review_mas_reciente_contiene_la_palabra_pizza = \
    antiguedades.join(usuarios_cuya_review_mas_reciente_contiene_la_palabra_pizza).reduce(lambda x, y: x if x[1][0][1] < y[1][0][1] else y)

In [24]:
anios_de_antiguedad_respecto_al_anio_actual = \
    antiguedades \
        .map(lambda row: 2022 - row[1][1].year) \
        .cache()

In [25]:
print("La antigüedad promedio de los usuarios (en años) es:", anios_de_antiguedad_respecto_al_anio_actual.mean())

La antigüedad promedio de los usuarios (en años) es: 7.71877981585226


In [26]:
print("El usuario más antiguo cuya review más reciente contiene la palabra pizza se llama", usuario_con_mas_antiguedad_cuya_review_mas_reciente_contiene_la_palabra_pizza[1][0][0])

El usuario más antiguo cuya review más reciente contiene la palabra pizza se llama Hafiz


# S36

In [27]:
users_with_more_than_10_reviews = rdd_review \
    .filter(lambda row: row.user_id and len(row.user_id) == 22 and row.business_id and len(row.business_id) == 22) \
    .map(lambda row: (row.user_id, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .filter(lambda row: row[1] > 10) \
    .cache()

user_with_businesses_reviewed = rdd_review \
    .filter(lambda row: row.user_id and len(row.user_id) == 22 and row.business_id and len(row.business_id) == 22) \
    .map(lambda row: (row.user_id, row.business_id)) \
    .cache()

En este paso el resultado del join que era de la forma (user_id, (user_review_count, reviewed_business_id)) lo transformo a la forma (business_id, (user_id, user_review_count))

In [28]:
a = users_with_more_than_10_reviews \
    .join(user_with_businesses_reviewed) \
    .map(lambda row: (row[1][1], (row[0], row[1][0]))) \
    .cache()

In [30]:
businesses_with_locations = rdd_business \
    .map(lambda row: (row.business_id, (row.latitude, row.longitude))) \
    .cache()

En este paso luego del join los registros resultan de la forma `(business_id, ((user_id, user_review_count), (business_latitude, business_longitude))`

In [31]:
c = a \
    .join(businesses_with_locations) \
    .cache()

In [33]:
# Acá en el primer map me quedo con el registro ((user_id, user_review_count), (latitude, longitude)).
# En el reduceByKey sumo las latitudes y las longitudes por una misma key (user_id, user_review_count).
# En el último map divido la sumatoria por el user_review_count para lograr el promedio y así obtener la ubicación del usuario.
users_locations = c \
    .map(lambda row: row[1]) \
    .reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1])) \
    .map(lambda row: (row[0][0], (row[1][0]/row[0][1], row[1][1]/row[0][1]))) \
    .cache()

In [35]:
# Los registros de éste RDD son de la forma ((user_id_1, (lat_1, long_1)), (user_id_2, (lat_2, long_2)))
cartesian_rdd_users_locations = users_locations.cartesian(users_locations).cache()

In [37]:
# Acá en el primer map transformo los registros a ((user_id_1, user_id_2), distance_between_user_1_and_user_2)).
# Luego filtro a los registros cuyos usuarios sean distintos (sino me queda el caso de user_id_1 con user_id_1 por culpa del cartesian).
# Luego me quedo con los más cercanos
closest_users = cartesian_rdd_users_locations \
    .map(lambda x: ((x[0][0], x[1][0]), distance(x[0][1][0], x[0][1][1], x[1][1][0], x[1][1][1]))) \
    .filter(lambda x: x[0][0] != x[0][1]) \
    .reduce(lambda x, y: x if x[1] < y[1] else y)

In [39]:
print(f"Los usuarios (que hicieron más de 10 reviews) más cercanos son (sus id): {closest_users[0][0]} y {closest_users[0][1]}")

Los usuarios (que hicieron más de 10 reviews) más cercanos son (sus id): xfSiMtuhrlZNszvU2q_bCQ y v8NMBO2ZyL_-sNL0uKneAQ
